In [1]:
import os
import re
import time
import numpy as np
import pandas as pd
from pathlib import Path
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver import ChromeOptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

In [87]:
filepath = Path("C:/Users/yym/Desktop/thalassemia peptidomics data analysis/rename Data")
filelist = list(filepath.glob("**/*peptide.xlsx"))
for f in filelist:
       
        ###################################文件读取部分################################### 
    name = os.path.basename(f)
    #读取excel去除NA，contaminant
    df = pd.read_excel(f)
    df = df.dropna(axis=0,subset = ['Master Protein Accessions']) 
    df = df[-df['Master Protein Accessions'].str.contains('contaminant')]
    
    #获取proteasix所需要的内容
    number,AC,start,end = [],[],[],[]
    sta_end = re.compile(r".*?\[(\d+)-(\d+)\].*?")
    for idx,row in df.iterrows():
        if ";" not in row["Positions in Master Proteins"]:
            AC.append(row["Master Protein Accessions"])
            start_pos = re.match(sta_end,row["Positions in Master Proteins"])
            s1, s2 = start_pos.group(1), start_pos.group(2)
            start.append(s1)
            end.append(s2)
        else:
            AC.append(row["Master Protein Accessions"].split(";")[0])
            start_pos = re.match(sta_end,row["Positions in Master Proteins"].split(";")[0])
            s1, s2 = start_pos.group(1), start_pos.group(2)
            start.append(s1)
            end.append(s2)
            
            
    #peptide 去重
    data_frame = pd.DataFrame({"AC":AC,"start":start,"end":end})
    data_frame = data_frame.drop_duplicates(subset=None, keep='first', inplace=False)

    #去重后的数据重新写入列表
    AC = data_frame["AC"].tolist()
    start = data_frame["start"].tolist()
    end = data_frame["end"].tolist()
    number = [x for x in range(1,len(AC))]
    
    
    
    #由于proteasix一次最多只能提交3000条肽段，如果文件中肽段超过，则需要分成多个部分，分别提交
    #根据肽段数量判断需要分成几部分
    part_num = len(AC)//3000 + 1
    #将每个部分需要的内容存入相应字典，字典关键词为“part1”,"part2"....
    part_AC,part_start,part_end,part_number = {},{},{},{}
    batch = 3000
    for i in range(1,part_num+1):
        exec("part_AC['part{}']=AC[batch*(i-1):batch*i]".format(i))
        exec("part_start['part{}']=start[batch*(i-1):batch*i]".format(i))
        exec("part_end['part{}']=end[batch*(i-1):batch*i]".format(i))
        exec("part_number['part{}']=number[batch*(i-1):batch*i]".format(i))

    
    
    filepart=1
    for part in part_AC.keys():
        number = part_number[part]
        AC = part_AC[part]
        start = part_start[part]
        end = part_end[part]
        #将proteasix所需要的文本拼接为字符串    
        string = str(number[0])+"\t"+AC[0]+"\t"+str(start[0])+"\t"+str(end[0])
        for i in range(len(number)-1):
            string = string+"\n"+str(number[i])+"\t"+AC[i]+"\t"+str(start[i])+"\t"+str(end[i])
            
            
            
        ###################################爬虫部分###################################
    
        #实例化option
        option = webdriver.ChromeOptions()
    
        #chrome启动位置
        option.binary_location = r"C:\Program Files\Google\Chrome\Application\chrome.exe"
        #option.add_experimental_option("debuggerAddress", "127.0.0.1:9999")
        desired_capabilities = DesiredCapabilities.CHROME  # 修改页面加载策略
        desired_capabilities["pageLoadStrategy"] = "none"  
    
        #chromedriver位置，传入option
        driver = Chrome(executable_path="D:/chromedriver.exe",options = option)   
    
        #URL
        url = "http://www.proteasix.org"
        driver.get(url)
    
        #由于#document==$0, 子frame被隐藏，需要找到子frame并进入才能找到XPATH
        frame = WebDriverWait(driver, 30).until(
                        EC.presence_of_element_located(
                            (By.CSS_SELECTOR, 'frame')))
        driver.switch_to.frame(frame)
        print("###########加载完成###########")
    
    
        #点击prediction tool的XPATH,进入工具
        driver.find_element(By.XPATH,'//*[@id=\"navigation\"]/ul/li[5]/a').click()
        print("#########进入prediction tool#########")
        time.sleep(1)
    
        #使用 Predicted: cleavage probability based on MEROPS工具
        driver.find_element(By.XPATH,'//*[@id="OPmode2"]/i').click()
        time.sleep(1)
    
        #用driver执行js代码将构造的string输入到文本框
        js = 'var ucode = document.getElementById("myTextAreaINPUT0"); ucode.value=arguments[0]'
        driver.execute_script(js,string)
        print("#########输入完成#########")
    
    
        #隐式等待每步执行完成，点击“next step”
        time.sleep(1)
        driver.find_element(By.XPATH,'//*[@id="fragment-1"]/a').click()
        driver.implicitly_wait(60)

        driver.find_element(By.XPATH,'//*[@id="fragment-2"]/a').click()
        driver.implicitly_wait(100)

        driver.find_element(By.XPATH,'//*[@id="fragment-3"]/a').click()
        print("#########分析完成#########")
  
        #由于直接点击Download无效，所以取出结果文本框内容(str格式)，写入到文本文件夹中
        result_str = driver.find_element(By.XPATH,'//*[@id="myTextAreaINPUT3"]').get_attribute("value")
    
    
        save_path = "C:\\Users\\yym\\Desktop\\thalassemia peptidomics data analysis\\rename Data\\A_Proteasix"
        file_name = re.match(r"\w\d+",name).group()
        full_path = save_path + "\\" + file_name + "_"+ str(filepart) + ".txt"
        with open(full_path, "w", encoding='utf-8') as f:
            f.write(result_str)
            f.close()
        
        driver.close()
        filepart += 1

C:\Users\yym\AppData\Local\Temp\ipykernel_11160\3368318724.py:76: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = Chrome(executable_path="D:/chromedriver.exe",options = option)


###########加载完成###########
#########进入prediction tool#########
#########输入完成#########
#########分析完成#########


C:\Users\yym\AppData\Local\Temp\ipykernel_11160\3368318724.py:76: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = Chrome(executable_path="D:/chromedriver.exe",options = option)


###########加载完成###########
#########进入prediction tool#########
#########输入完成#########


KeyboardInterrupt: 